<a href="https://colab.research.google.com/github/iqbalamo93/Logistic/blob/master/Logistic_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
url='https://dsserver-prod-resources-1.s3.amazonaws.com/24/auto.csv'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
pd.options.display.max_columns=255


In [0]:
cars = pd.read_csv(url)
unique_regions = cars["origin"].unique()


In [4]:
unique_regions

array([1, 3, 2])

In [5]:
cars.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [6]:
cars['year'].value_counts()

73    40
78    36
76    34
82    30
75    30
79    29
70    29
81    28
77    28
72    28
80    27
71    27
74    26
Name: year, dtype: int64

In [7]:
dummy_cylinders = pd.get_dummies(cars["cylinders"], prefix="cyl")
cars = pd.concat([cars, dummy_cylinders], axis=1)
dummy_years = pd.get_dummies(cars["year"], prefix="year")
cars = pd.concat([cars, dummy_years], axis=1)
cars = cars.drop("year", axis=1)
cars = cars.drop("cylinders", axis=1)
print(cars.head())

    mpg  displacement  horsepower  weight  acceleration  origin  cyl_3  cyl_4  \
0  18.0         307.0       130.0  3504.0          12.0       1      0      0   
1  15.0         350.0       165.0  3693.0          11.5       1      0      0   
2  18.0         318.0       150.0  3436.0          11.0       1      0      0   
3  16.0         304.0       150.0  3433.0          12.0       1      0      0   
4  17.0         302.0       140.0  3449.0          10.5       1      0      0   

   cyl_5  cyl_6  cyl_8  year_70  year_71  year_72  year_73  year_74  year_75  \
0      0      0      1        1        0        0        0        0        0   
1      0      0      1        1        0        0        0        0        0   
2      0      0      1        1        0        0        0        0        0   
3      0      0      1        1        0        0        0        0        0   
4      0      0      1        1        0        0        0        0        0   

   year_76  year_77  year_78  ye

In [8]:
cars.head()

,mpg,displacement,horsepower,weight,acceleration,origin,cyl_3,cyl_4,cyl_5,cyl_6,cyl_8,year_70,year_71,year_72,year_73,year_74,year_75,year_76,year_77,year_78,year_79,year_80,year_81,year_82
0,18.0,307.0,130.0,3504.0,12.0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,15.0,350.0,165.0,3693.0,11.5,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,18.0,318.0,150.0,3436.0,11.0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
3,16.0,304.0,150.0,3433.0,12.0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4,17.0,302.0,140.0,3449.0,10.5,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
shuffled_rows  =  np.random.permutation(cars.index)
shuffled_cars = cars.iloc[shuffled_rows]

In [0]:
shuffled_rows = np.random.permutation(cars.index)
shuffled_cars = cars.iloc[shuffled_rows]
highest_train_row = int(cars.shape[0] * .70)
train = shuffled_cars.iloc[0:highest_train_row]
test = shuffled_cars.iloc[highest_train_row:]

In [11]:
shuffled_cars.loc[0:highest_train_row,['displacement','acceleration']]

,displacement,acceleration
0,307.0,12.0
288,351.0,14.3
356,168.0,12.6
265,134.0,14.2
171,119.0,17.0
...,...,...
117,116.0,15.5
299,105.0,13.2
121,121.0,14.0
120,318.0,11.0


In the one-vs-all approach, we're essentially converting an n-class (in our case n is 3) classification problem into n binary classification problems. For our case, we'll need to train 3 models:

A model where all cars built in North America are considered Positive (1) and those built in Europe and Asia are considered Negative (0).
A model where all cars built in Europe are considered Positive (1) and those built in North America and Asia are considered Negative (0).
A model where all cars built in Asia are labeled Positive (1) and those built in North America and Europe are considered Negative (0).
Each of these models is a binary classification model that will return a probability between 0 and 1. When we apply this model on new data, a probability value will be returned from each model (3 total). For each observation, we choose the label corresponding to the model that predicted the highest probability.

We'll use the dummy variables we created from the cylinders and year columns to train 3 models using the LogisticRegression class from scikit-learn.

In [0]:
from sklearn.linear_model import LogisticRegression

unique_origins = cars["origin"].unique()
unique_origins.sort()

models = {}

In [13]:
cars["origin"].unique()


array([1, 3, 2])

In [0]:
cars["origin"].unique().sort()

In [0]:
features = [c for c in train.columns if c.startswith("cyl") or c.startswith("year")]

for origin in unique_origins:
    model = LogisticRegression()
    
    X_train = train[features]
    y_train = train["origin"] == origin

    model.fit(X_train, y_train)
    models[origin] = model

In [16]:
models

{1: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 2: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 3: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=Fal

In [0]:
testing_probs = pd.DataFrame(columns=unique_origins)  

for origin in unique_origins:
    # Select testing features.
    X_test = test[features]   
    # Compute probability of observation being in the origin.
    testing_probs[origin] = models[origin].predict_proba(X_test)[:,1]

In [18]:
testing_probs

,1,2,3
0,0.898896,0.072305,0.032726
1,0.390384,0.223401,0.358604
2,0.969056,0.015841,0.031542
3,0.964060,0.033430,0.015891
4,0.300195,0.540543,0.160821
...,...,...,...
113,0.362921,0.300292,0.319182
114,0.887852,0.049132,0.061020
115,0.390384,0.223401,0.358604
116,0.302888,0.379401,0.308650


In [0]:
X_train = train[features]
y_train = train["origin"] == 1

In [20]:
X_train.shape,y_train.shape



((274, 18), (274,))

In [21]:
train["origin"] 

308    3
153    1
180    2
20     2
36     1
      ..
319    3
110    3
46     1
324    2
360    1
Name: origin, Length: 274, dtype: int64

In [22]:
y_train

308    False
153     True
180    False
20     False
36      True
       ...  
319    False
110    False
46      True
324    False
360     True
Name: origin, Length: 274, dtype: bool

In [23]:
predicted_origins = testing_probs.idxmax(axis=1)
print(predicted_origins)

0      1
1      1
2      1
3      1
4      2
      ..
113    1
114    1
115    1
116    2
117    1
Length: 118, dtype: int64


In [24]:
testing_probs.head()

,1,2,3
0,0.898896,0.072305,0.032726
1,0.390384,0.223401,0.358604
2,0.969056,0.015841,0.031542
3,0.964060,0.033430,0.015891
4,0.300195,0.540543,0.160821


In [25]:
features

['cyl_3',
 'cyl_4',
 'cyl_5',
 'cyl_6',
 'cyl_8',
 'year_70',
 'year_71',
 'year_72',
 'year_73',
 'year_74',
 'year_75',
 'year_76',
 'year_77',
 'year_78',
 'year_79',
 'year_80',
 'year_81',
 'year_82']